In [1]:
import os
import requests
import zipfile

In [2]:
cwd = os.getcwd() # current working directory
print(cwd)

/python-fhe/examples


In [3]:
data_dir = os.path.join(cwd, "datasets")
if os.path.exists(data_dir):
    pass
else:
    os.mkdir(data_dir)
print(data_dir)

/python-fhe/examples/datasets


In [4]:
mnist_zip = os.path.join(data_dir, "mnist.zip")
if os.path.exists(mnist_zip):
    print("Skipping mnist download")
else:
    print("Downloading Fashion-MNIST")
    mnist_url = "http://nextcloud.deepcypher.me/s/wjLa6YFw8Bcbra9/download"
    r = requests.get(mnist_url, allow_redirects=True, verify=False)
    with open(mnist_zip, "wb") as f:
        f.write(r.content)

Skipping mnist download


In [5]:
unzip_dir = os.path.join(data_dir, "mnist")
if os.path.exists(unzip_dir):
    pass
else:
    os.mkdir(unzip_dir)
with zipfile.ZipFile(mnist_zip, "r") as zip_ref:
    zip_ref.extractall(unzip_dir)

In [6]:
import pandas as pd
import numpy as np
import tqdm

In [7]:
train_file = os.path.join(unzip_dir, "fashion-mnist_train.csv") 
test_file = os.path.join(unzip_dir, "fashion-mnist_test.csv")
train = pd.read_csv(train_file)
test = pd.read_csv(train_file)
# train

In [8]:
train_y = train.iloc[:, 0]
train_x = train.iloc[:, 1:]
test_x = train.iloc[:, 1:]
test_y = test.iloc[:, 0]
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
test_x = test_x.to_numpy()
test_y = test_y.to_numpy()
print(train_x.shape)
print(train_y.shape)

(60000, 784)
(60000,)


In [9]:
import seal # https://github.com/Huelse/SEAL-Python OR https://github.com/DreamingRaven/python-seal
from fhe.nn.layer.cnn import Layer_CNN # from this library
from fhe.nn.layer.ann import Layer_ANN # from this library
from fhe.rearray import ReArray # meta encryption object from this library

In [10]:
encryption_parameters = {
            "scheme": seal.scheme_type.CKKS,
            "poly_modulus_degree": 8192*2,
            "coefficient_modulus":
                [45, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 45],
            "scale": pow(2.0, 30),
            "cache": True,
}

In [11]:
# Generate Encrypted data peace-meal (as it can get very large)
def row_encrypted_generator(data: np.ndarray, shape: tuple):
    """Generate encrypted data of desired shape from rows."""
    for row in data:
        row = np.reshape(row, newshape=shape) / 255 # reshape to image shape and normalise between 0-1
        yield ReArray(row, **encryption_parameters)

In [12]:
cnn = Layer_CNN(weights=( 1, 6, 6 ), stride=[ 1, 2, 2 ], bias=0)
# for cyphertext in row_encrypted_generator(data=train_x, shape=( 1, 28, 28 )):
#     cnn_acti = cnn.forward(cyphertext)
#     if dense is None:
#         dense = Layer_ANN(weights=(len(cnn_acti),), bias=0)
#     dense.forward(cnn_acti)